**MODEL**

In [ ]:
!pip install datasets

In [ ]:
# import zipfile
# import urllib.request
# import os

In [ ]:
import torch
import torchaudio
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor


In [ ]:
!wget https://www.openslr.org/resources/43/ne_np_female.zip
!unzip ne_np_female.zip
!ls ne_np_female

In [ ]:
colnames=['path','sentence']
df  = pd.read_csv('/content/ne_np_female/line_index.tsv',sep='\\t',header=None,names = colnames)
df['path'] = '/content/ne_np_female/wavs/'+df['path'] +'.wav'

<ipython-input-7-f324ee143afd>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df  = pd.read_csv('/content/ne_np_female/line_index.tsv',sep='\\t',header=None,names = colnames)


In [ ]:
train, test = train_test_split(df, test_size=0.1)

In [ ]:
test.to_csv('/content/ne_np_female/line_index_test.csv')


In [ ]:
test_dataset = load_dataset('csv', data_files='/content/ne_np_female/line_index_test.csv',split = 'train')


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("SumitMdhr/fASR-pr")
model = Wav2Vec2ForCTC.from_pretrained("SumitMdhr/model-13.58")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/2.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [ ]:
def speech_file_to_array_fn(batch):
  speech_array, sampling_rate = torchaudio.load(batch["path"])
  batch["speech"] = resampler(speech_array).squeeze().numpy()
  return batch

In [ ]:
test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)

Map:   0%|          | 0/207 [00:00<?, ? examples/s]

In [ ]:
with torch.no_grad():
  logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
predicted_ids = torch.argmax(logits, dim=-1)

In [ ]:
print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["sentence"][:2])

Prediction: ['सङ्कर्सिङ बित्रको जन्म सुदुरपश्चिम नेपालको दार्चुला जिल्लामा भएको हो', 'उन्नाइसै उननसटरीमा बेजिङ्गयका बोसले च्रिनिया कमयुनिसबाटीबाट नौ बुन्दे सुझाव नक्सलबादरी आन्दोलनको इटिहासमा उलेखनीय छ']
Reference: ['शङ्कर सिंह बिस्टको जन्म सुदूरपश्चिम नेपालको दार्चुला जिल्लामा भएको हो', 'मउन्नाइस सय उनान्सत्तरीमा बेइजिङ गएका बोसले चिनियाँ कम्युनिष्ट पार्टीबाट नौ बुँदे सुझाव नक्सलवादी आन्दोलनको इतिहासमा उल्लेखनीय छ']


# **SINGLE AUDIO MODEL TESTING**

In [ ]:
def predict_audio(path):
  speech_array, sampling_rate = torchaudio.load(path)
  resampler = torchaudio.transforms.Resample(sampling_rate, 16_000)
  speech = resampler(speech_array).squeeze().numpy()
  inputs = processor(speech, sampling_rate=16_000, return_tensors="pt", padding=True)
  with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
  predicted_ids = torch.argmax(logits, dim=-1)
  return processor.batch_decode(predicted_ids)

In [ ]:
print(predict_audio('/content/Tabo Marg 4.m4a'))

['']
